## *Variables Générales et Imports*

In [44]:
import urllib.request
import os
import json
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.cluster import KMeans
from PIL import Image

DATA_JSON_PATH = "data.json"
USERS_JSON_PATH = "users.json"

IMAGE_DATATYPES = ["label","date","model","colors","orientation","taille","tags"]
USERS_PREFSTYPES = ["id","colors","orientation","sizes","tags","imagesLiked"]
IMAGE_SIZES = ["Grande","Moyenne","Petite","Vignette"]

# Implémentation du Système de Recommandation

## Collecte de données

In [45]:
def CollecteDesImages(queryLimit : str):
    # Récupération de la Query
    url = "https://query.wikidata.org/sparql?query=SELECT%20%3Fitem%20%3FitemLabel%20%3Fimage%0AWHERE%0A%7B%0A%3Fitem%20wdt%3AP31%20wd%3AQ144%20.%0A%3Fitem%20wdt%3AP18%20%3Fimage%0ASERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22%20%7D%0A%7D%0ALIMIT%20"+str(queryLimit)+"&format=json"
    response = urllib.request.urlopen(url)
    responseJSON  = json.load(response)

    # Création de la dataframe
    columns   = ["label","format","link","path"]
    dataArray = []
    for data in responseJSON["results"]["bindings"]:
        link = data["image"]["value"]
        label = data["itemLabel"]["value"]
        format = str(link).split(".")[-1]
        path = "./images/" + label +"."+ format
        dataArray.append([label,format,link,path])

    dataframe = pd.DataFrame(dataArray, columns= columns)


    # Vidage du dossier images
    files = glob.glob('./images/*')
    for f in files:
        os.remove(f)

    sizes = []
    # Telechargement des images
    length = len(dataframe.index)
    for i in range(length):
        link = dataframe._get_value(i,'link')
        path = dataframe._get_value(i,'path')
        urllib.request.urlretrieve(link, path)

        img = Image.open(path)
        sizes.append(np.array(img.size))
        img.close()

    dataframe["résolution"] = sizes

    # Création du fichier JSON 
    file = open(DATA_JSON_PATH, "w") 
    file.write(dataframe.to_json())
    file.close()


def RecuperationMetadonneesEXIF():
    dataframe = pd.read_json(DATA_JSON_PATH)

    tags = ["label","date","model"]
    tagsWantedEXIF = [306,272]
    dataEXIF = []
        
    length = len(dataframe.index)
    for i in range(length):
        path = dataframe._get_value(i,'path')
            
        image = Image.open(path)
        exif = image.getexif()

            

        array = [dataframe._get_value(i,'label')]
        for tag in tagsWantedEXIF:
            array.append(exif.get(tag))
        dataEXIF.append(array)
        
    dataframeEXIF = pd.DataFrame(dataEXIF, columns=tags)
    for c in dataframeEXIF.columns :
        dataframe[c] = dataframeEXIF[c]
            
    file = open(DATA_JSON_PATH, "w")
    file.write(dataframe.to_json())
    file.close()

def getDataDataFrame():
    return pd.read_json(DATA_JSON_PATH)



def CollecteDesDonnees(queryLimit : str):
    CollecteDesImages(queryLimit)
    RecuperationMetadonneesEXIF()

## Étiquetage et annotation

In [46]:
def CalculerCouleursPredominantes(Ndiv, Res):
    Ndiv = 3
    Res = 100

    dataframe = pd.read_json(DATA_JSON_PATH)

    colors = []
    distributions = []

    for path in dataframe.path:
        img = Image.open(path).convert("RGB")
        imgArray = np.array(img.getdata())[::Res]
        img.close()
        imgDataframe = pd.DataFrame(imgArray, columns=["red","green","blue"])

        model=KMeans(n_clusters= Ndiv, n_init='auto')
        model.fit(imgDataframe)

        distribution = []
        total = len(list(model.labels_))
        for i in range(Ndiv):
            distribution.append(list(model.labels_).count(i)/total)
        distributions.append(distribution)

        meansList = (np.floor(model.cluster_centers_)).tolist()

        colors.append(meansList)

    dataframe["colors"] = colors
    dataframe["colorsDistribution"] = distributions

    # Création du fichier JSON 
    colorTable = open(DATA_JSON_PATH, "w")
    colorTable.write(dataframe.to_json())
    colorTable.close()


def ImageCouleursPredominantesAff(imageIndex):
    dataDataFrame = pd.read_json()
    colors = np.array(dataframe["colors"][imageIndex])/255
    distribution = np.array(dataframe["colorsDistribution"][0])*100

    fig = plt.figure()
    ax = fig.add_subplot()

    ax.pie(distribution, colors = colors)


def CalculerOrientationEtTaille():
    dataframe = pd.read_json(DATA_JSON_PATH)
    orientations = []
    tailles = []
    for taille in dataframe["résolution"]:
        # Orientation
        if taille[0] > taille[1]:
            orientations.append("Paysage")
        else:
            orientations.append("Portrait")
        
        nbrPixel = taille[0] * taille[1]
    
        if   nbrPixel >= 1920*1080:
            tailles.append("Grande")
        elif nbrPixel >= 1280*720:
            tailles.append("Moyenne")
        elif nbrPixel >= 720*480:
            tailles.append("Petite")
        else :
            tailles.append("Vignette")
        

    dataframe["orientation"] = orientations
    dataframe["taille"] = tailles

    # Création du fichier JSON 
    colorTable = open(DATA_JSON_PATH, "w")
    colorTable.write(dataframe.to_json())
    colorTable.close()


def placeHolder():
    dataframe = pd.read_json(DATA_JSON_PATH)
    dataframe["tags"] = [['dog'] for i in range(len(dataframe["label"]))]

    colorTable = open(DATA_JSON_PATH, "w")
    colorTable.write(dataframe.to_json())
    colorTable.close()


def EtiquetageEtAnnotation():
    CalculerCouleursPredominantes(3,100)
    CalculerOrientationEtTaille()
    placeHolder()



## Analyse des données

In [70]:
def getUserDataFrame():
    return pd.read_json(USERS_JSON_PATH)

def createUser(userId):
    columns = USERS_PREFSTYPES
    row = [[userId,[],None,[],[],[]]]
    newData = pd.DataFrame(row, columns= columns)

    try :
        userDataFrame = getUserDataFrame()
    except FileNotFoundError :
        userDataFrame = resetUsersDataFrame()
        
    if not (userId in list(userDataFrame["id"])):
        userDataFrameChanged = pd.concat([userDataFrame,newData],ignore_index=True)
        saveUserDataFrame(userDataFrameChanged)
    else :
        print("User already exist")

def deleteUser(userId):
    userDataFrame = getUserDataFrame()

    if userId in list(userDataFrame["id"]) :
        if userId in list(userDataFrame["id"]):
            index = list(userDataFrame["id"]).index(userId)
            userDataFrame = userDataFrame.drop([index])
        saveUserDataFrame(userDataFrame)

def resetUsersDataFrame():
    columns = USERS_PREFSTYPES
    userDataFrame = pd.DataFrame(columns = columns)
    saveUserDataFrame(userDataFrame)
    return userDataFrame

def getUserPreferences(userId):
    userDataFrame = getUserDataFrame()
    prefs = []
    if userId in list(userDataFrame["id"]):
        index = list(userDataFrame["id"]).index(userId)
        prefs = userDataFrame.iloc[index]
    return(prefs)

def saveUserDataFrame(userDataFrame):
    usersJSON = open(USERS_JSON_PATH, "w")
    usersJSON.write(userDataFrame.to_json())
    usersJSON.close()

def ajouterImagesUserManuellement(userId):
    dataDataFrame = pd.read_json(DATA_JSON_PATH)
    userDataFrame = pd.read_json(USERS_JSON_PATH)

    i = random.randint(0,len(dataDataFrame["label"])-1)
    while i in userDataFrame["imagesLiked"][list(userDataFrame["id"]).index(userId)] :
        i = random.randint(0,len(dataDataFrame["label"])-1)

    img = Image.open(dataDataFrame["path"][i]).convert('RGB')
    img.show()

    invalid = True
    while invalid:
        ans = input("Do you like this image ? [y/n]").capitalize()
        invalid = ans not in ["Y","N"]
    liked = ans == "Y"  
    if liked :
        (userDataFrame["imagesLiked"][list(userDataFrame["id"]).index(userId)]).append(i)

    saveUserDataFrame(userDataFrame)
    userDataFrame

def ajouterImagesUserAuto(userId, nbrImages):
    dataDataFrame = pd.read_json(DATA_JSON_PATH)
    userDataFrame = pd.read_json(USERS_JSON_PATH)
    userIndex = list(userDataFrame["id"]).index(userId)

    imagesLikedBefore = list(userDataFrame["imagesLiked"][userIndex])

    imagesIndex = []
    for n in range(nbrImages):
        i = random.randint(0,len(dataDataFrame["label"])-1)
        while i in (userDataFrame["imagesLiked"][userIndex] or imagesIndex) :
            i = random.randint(0,len(dataDataFrame["label"])-1)
        imagesIndex.append(i)

    userDataFrame["imagesLiked"][userIndex] += imagesIndex

    saveUserDataFrame(userDataFrame)

    imagesLikedNow = list(userDataFrame["imagesLiked"][userIndex])
    print(f'Images likées : {[image for image in imagesLikedNow if image not in imagesLikedBefore]}')

def deleteUserLikedImages(userId):
    userDataFrame = pd.read_json(USERS_JSON_PATH)
    userIndex = list(userDataFrame["id"]).index(userId)
    userDataFrame["imagesLiked"][userIndex] = []
    saveUserDataFrame(userDataFrame)

def CalculPreferencesUser():
    dataDataFrame = pd.read_json(DATA_JSON_PATH)
    userDataFrame = pd.read_json(USERS_JSON_PATH)

    for userId in list(userDataFrame["id"]):
        print(userId)
        index = list(userDataFrame["id"]).index(userId)
        imagesLikedindex = list(userDataFrame["imagesLiked"][index])

        colors = []
        orientations = []
        sizes = []
        tags = []

        if imagesLikedindex != []:

            # Collecte des Informations des Images Likées
            for i in imagesLikedindex:
                colors += list(dataDataFrame["colors"])[i]
                orientations += list(dataDataFrame["orientation"])[i]
                sizes.append(list(dataDataFrame["taille"])[i])
                tags += list(dataDataFrame["tags"])[i]

            # Couleurs préférées (Les couleurs majoritaires des images)
            userDataFrame["colors"][index] = colors


            # Orientation préféré (Celle majoritaire)
            if orientations.count("Paysage") > len(orientations)//2 :
                userDataFrame["orientation"][index] = "Paysage"
            else:
                userDataFrame["orientation"][index] = "Portrait"

            # Tailles préférées (les 2 majoritairement représentées)
            t1 = 0
            t2 = 0
            for i,taille in enumerate(IMAGE_SIZES) :
                if (c := sizes.count(taille)) > t1 :
                    t1 = i
                elif (c := sizes.count(taille)) > t2 and c != t1:
                    t2 = i
            userDataFrame["sizes"][index] = [IMAGE_SIZES[t1],IMAGE_SIZES[t2]]

            # Tags préférées (non implémenté)
            userDataFrame["tags"][index] = []

    saveUserDataFrame(userDataFrame)


## Visualisation des données

In [48]:
def visualistionDesDonnees(target, category):
     dataDataFrame = pd.read_json(DATA_JSON_PATH)
     userDataFrame = pd.read_json(USERS_JSON_PATH)

     df = dataDataFrame
     userIds = list(userDataFrame["id"])
     target = ""
     if target in userIds:
          df = pd.DataFrame()
          for imageIndex in userDataFrame["imagesLiked"]:
               df = pd.concat([df, dataDataFrame.iloc[imageIndex]])

     distribution = []

     for item in df[category]:
          if (item != None) and (item != []):
               if category == 'date':
                         distribution.append(str(item).split(':')[0])
               else:
                    distribution.append(str(item))
          else :
               distribution.append("Indéterminé")

     plt.hist(distribution, rwidth = 0.8, color = "orange")

def caracteristiquesDesCouleurs():
     dataDataFrame = pd.read_json(DATA_JSON_PATH)
     category = "colors"
     distribution = []

     for item in dataDataFrame[category]:
          if (item != None) and (item != []):
               distribution += item
          else :
               distribution.append("Indéterminé")
     distribution = np.array(distribution)

     r = [c[0]/255  for c in distribution]
     g = [c[1]/255  for c in distribution]
     b = [c[2]/255  for c in distribution]
     c = distribution/255

     fig = plt.figure()
     ax = fig.add_subplot(projection='3d')
     ax.scatter(r,g,b,c = c)

## Système de Recommandation

In [49]:
def recommand():
    dataDataFrame = pd.read_json(DATA_JSON_PATH)

    userId = ""
    while not userId in list(userDataFrame["id"]) :
        userId = input("user Id :")
    userPrefs = getUserPreferences(userId)
    recommandation = pd.DataFrame()

    if list(userPrefs) != [] :
        
        if userPrefs['sizes'] != [] :
            for size in userPrefs['sizes']:
                recommandation = pd.concat([recommandation,dataDataFrame.loc[dataDataFrame['taille'] == size]])

        if userPrefs['orientation'] != None :
            recommandation = recommandation.loc[recommandation['orientation'] == userPrefs['orientation']]
            
    else:
        recommandation = dataDataFrame
        
    return recommandation.sort_index()


# Tests

In [50]:
CollecteDesDonnees(queryLimit= 10)

In [51]:
EtiquetageEtAnnotation()

In [ ]:
getDataDataFrame()

In [ ]:
visualistionDesDonnees(target= "all", category= "taille")

In [ ]:
caracteristiquesDesCouleurs()

In [ ]:
userId = 'john'
createUser(userId)
ajouterImagesUserAuto(userId, nbrImages=5)

In [71]:
CalculPreferencesUser()
getUserDataFrame()

User already exist
Images likées : [1, 1, 9, 8, 9]
john


,id,colors,orientation,sizes,tags,imagesLiked
0,john,"[[139.0, 141.0, 134.0], [201.0, 212.0, 208.0],...",Portrait,"[Vignette, Grande]",[],"[3, 5, 6, 0, 7, 1, 1, 9, 8, 9]"


In [61]:
deleteUser("john")

In [68]:
getUserDataFrame()

,id,colors,orientation,sizes,tags,imagesLiked


In [67]:
resetUsersDataFrame()

,id,colors,orientation,sizes,tags,imagesLiked
